In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from src.utils import utils

PROJECT_DIR = os.getcwd()

In [7]:
# editable part
dataset_name = "milk_stage2"
run_name = "run3"
dataset_path = f"{PROJECT_DIR}/dataset/{dataset_name}/result/{run_name}"

In [3]:
class Tools:
    def __init__(self):
        pass

    @staticmethod
    def reproject_ray(intrinsic, extrinsic, uv):
        # extract oriagin and direction
        origin = extrinsic[:3, 3]
        fx, fy, cx, cy = intrinsic[0, 0], intrinsic[1, 1], intrinsic[0, 2], intrinsic[1, 2]
        norm_coor = np.array([(uv[0] - cx) / fx, - (uv[1] - cy) / fy, 1])
        direction = extrinsic[:3, :3].dot(norm_coor / np.linalg.norm(norm_coor))
        return origin, direction

    @staticmethod
    def rotation_matrix_to_rodriguez(R):
        # R = inv_proj_P2[:3, :3]
        angle = np.arccos((np.trace(R) - 1) / 2) + 1.e-10
        axis = 1 / (2 * np.sin(angle)) * np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

        # Calculate Rodriguez rotation vector
        rodriguez_vector = angle * axis
        return rodriguez_vector

    @staticmethod
    def rodriguez_to_rotation_matrix(rodriguez_vector):
        # Calculate angle of rotation and unit axis vector
        angle = np.linalg.norm(rodriguez_vector)
        axis = rodriguez_vector / angle if angle != 0 else np.array([0, 0, 0])

        # Rodrigues' rotation formula
        skew_symmetric = np.array([[0, -axis[2], axis[1]],
                                [axis[2], 0, -axis[0]],
                                [-axis[1], axis[0], 0]])
        rotation_matrix = np.eye(3) + np.sin(angle) * skew_symmetric + (1 - np.cos(angle)) * np.dot(skew_symmetric, skew_symmetric)
        return rotation_matrix
    

### Generate images from specified dataset and run name

In [93]:
%matplotlib agg

for i in range(35, len(glob.glob(os.path.join(dataset_path, "record_*.txt")))):
    # camera_intrinsic = np.array([[931.81027925, 0. , 960.], [0. ,931.81027925, 540.], [0. ,0. ,1.]])
    camera_intrinsic = np.array([[931.81027925, 0. , 960.], [0. ,931.81027925, 540.], [0. ,0. ,1.]])
        
    # --- read training data
    training_data = np.array(utils.read_txt(os.path.join(dataset_path, f"record_{i}.txt"))[0][1:-1].split(", ")).astype(float)
    num_camera, num_pts, data = int(training_data[0]), int(training_data[1]), training_data[2:]
    param_camera_temp = data[:6*num_camera].reshape(-1, 6)
    param_pts = data[6*num_camera:].reshape(-1, 3)

    param_camera = [np.eye(4) for i in param_camera_temp]
    param_camera_origin_direction = [np.zeros(12) for i in param_camera_temp]
    for j, cam_rodrig in enumerate(param_camera_temp):
        param_camera[j][:3, 3] = cam_rodrig[:3]
        param_camera[j][:3, :3] = Tools.rodriguez_to_rotation_matrix(cam_rodrig[3:])
        # ray = Tools.reproject_ray(camera_intrinsic, param_camera[j], (960, 540))
        param_camera_origin_direction[j][:3] = cam_rodrig[:3]
        # param_camera_origin_direction[j][3:6] = ray[1]
        # plot x y z
        dir_x, dir_y, dir_z = np.array([1, 0, 0, 1]), np.array([0, 1, 0, 1]), np.array([0, 0, 1, 1])
        param_camera_origin_direction[j][3:6] = param_camera[j].dot(dir_x)[:3]
        param_camera_origin_direction[j][6:9] = param_camera[j].dot(dir_y)[:3]
        param_camera_origin_direction[j][9:12] = param_camera[j].dot(dir_z)[:3]
        
    param_camera = np.array(param_camera)
    param_camera_origin_direction = np.array(param_camera_origin_direction)

    # --- read known data
    known_data = utils.import_json(os.path.join(dataset_path, f"record_{i}.json"))
    known_camera = np.array(known_data["camera"]).reshape(-1, 4, 4)
    known_pts = np.array(known_data["pts"]).reshape(-1, 3)

    fixed_camera_origin_direction = [np.zeros(12) for i in known_camera]
    for j, ext_cam in enumerate(known_camera):
        # ray = Tools.reproject_ray(camera_intrinsic, ext_cam, (960, 540))
        fixed_camera_origin_direction[j][:3] = ext_cam[:3, 3]
        # fixed_camera_origin_direction[j][3:] = ray[1]
        dir_x, dir_y, dir_z = np.array([1, 0, 0, 1]), np.array([0, 1, 0, 1]), np.array([0, 0, 1, 1])
        fixed_camera_origin_direction[j][3:6] = ext_cam.dot(dir_x)[:3]
        fixed_camera_origin_direction[j][6:9] = ext_cam.dot(dir_y)[:3]
        fixed_camera_origin_direction[j][9:12] = ext_cam.dot(dir_z)[:3]

    fixed_camera_origin_direction = np.array(fixed_camera_origin_direction)

    ax = plt.figure(figsize=(15, 10)).add_subplot(projection='3d')

    # is_training_step_1 = len(param_camera_origin_direction) == 2
    is_training_step_1 = True

    # use ratio from the 2nd camera
    # ratio = 1 / np.linalg.norm(fixed_camera_origin_direction[1][:3])
    # param_pts *= ratio
    # param_camera_origin_direction *= ratio
    # known_pts *= ratio
    # fixed_camera_origin_direction *= ratio

    # training - scatter pt and cam
    ax.scatter(xs=param_pts[:, 0], ys=param_pts[:, 1], zs=param_pts[:, 2], s=3, zdir='y', label='Training pts', c='magenta')
    ax.scatter(xs=param_camera_origin_direction[:, :3][:, 0], ys=param_camera_origin_direction[:, :3][:, 1], zs=param_camera_origin_direction[:, :3][:, 2], zdir='y', label='Training camera', c='r')
    ax.scatter(xs=known_pts[:, 0], ys=known_pts[:, 1], zs=known_pts[:, 2], zdir='y', s=3, label='Known pts', c='cyan')

    # training - plot cam

    color = ["r", "g", "b"]
    arrow_ratio = 0.5
    num_camera_training = len(param_camera_origin_direction)
    for cam_data_param in param_camera_origin_direction:
        for k_idx, k in enumerate(cam_data_param[3:].reshape(3, 3)):
            start = cam_data_param[:3]
            end = k / np.linalg.norm(k - start)# / np.linalg.norm(k - start)#cam_data[:3] + cam_data[3:]
            delta = end - start
            end = start + arrow_ratio * delta
            poses = np.array([start, end]).reshape(-1, 3)
            ax.plot(xs=poses[:, 0], ys=poses[:, 1], zs=poses[:, 2], zdir="y", c=color[k_idx])

    if is_training_step_1:
        fixed_camera_origin_direction = fixed_camera_origin_direction[:-num_camera_training + 1]
        ax.scatter(xs=fixed_camera_origin_direction[:, :3][:, 0], ys=fixed_camera_origin_direction[:, :3][:, 1], zs=fixed_camera_origin_direction[:, :3][:, 2], zdir='y', label='Known camera', c='g')
        # known - plot cam
        for cam_data_fixed in fixed_camera_origin_direction:
            for k_idx, k in enumerate(cam_data_fixed[3:].reshape(3, 3)):
                start = cam_data_fixed[:3]
                end = k / np.linalg.norm(k - start)# / np.linalg.norm(k)#cam_data[:3] + cam_data[3:]
                delta = end - start
                end = start + arrow_ratio * delta
                # end = cam_data[:3] + cam_data[3:6]
                poses = np.array([start, end]).reshape(-1, 3)
                ax.plot(xs=poses[:, 0], ys=poses[:, 1], zs=poses[:, 2], zdir="y", c=color[k_idx])
    else:
        ax.scatter(xs=[], ys=[], zs=[], zdir='y', label='Known camera', c='g')


    # print(param_camera_origin_direction, fixed_camera_origin_direction)
    # print()

    # save figure
    img_output_filename = os.path.join(dataset_path, f"record_{i}.jpg")
    ax.view_init(elev=30., azim=-45, roll=0)
    ax.set_xlabel("X")
    ax.set_ylabel("Z")
    ax.set_zlabel("Y")
    # ax.set_xlim(-1.25, 1.25)
    # ax.set_ylim(-0.5, 2)
    # ax.set_zlim(0, 1.5)
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-0.5, 4.5)
    ax.set_zlim(0, 5)
    # ax.set_xlim(-5, 5)
    # ax.set_ylim(-1, 9)
    # ax.set_zlim(0, 10)

    ax.legend()
    ax.figure.savefig(img_output_filename)

    # plt.clf()
    # plt.gcf().clear()
    plt.close()

    # if i == 100:
    #     break

### Generate video file from images

In [8]:
max_frame = 324
imgs = [os.path.join(dataset_path, f"record_{i}.jpg") for i in np.arange(len(glob.glob(os.path.join(dataset_path, "*.jpg"))))[:max_frame]][::1]

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(f'output_{dataset_name}.avi', fourcc, 30.0, (1500, 1000))

for im_path in imgs[:]:
    im = cv2.imread(im_path)
    # im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    print(im_path, os.path.exists(im_path))

    out.write(im)
out.release()


/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_0.jpg True
/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_1.jpg True
/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_2.jpg True
/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_3.jpg True
/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_4.jpg True
/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_5.jpg True
/home/yudzyoga/Documents/Courses/2ndSemester/3DCV/Assignments/github/3dcv_group_10/Project/dataset/milk_stage2/result/run3/record_6.jpg True
/home/yudzyog